In [3]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 24.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=ad2e3f9a0866cb78f4beb45fecdc3d77721a637ab01ece27b5a06107460c249c
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [4]:
import dask
import dask.dataframe as dd

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import base64
import cv2
from PIL import Image
import io

import tensorflow as tf
from tensorflow import keras

from pyspark.sql import SparkSession
import pyspark
spark = SparkSession.builder.getOrCreate()

In [5]:
tf.__version__

'2.9.2'

In [6]:
keras.__version__

'2.9.0'

In [7]:
def stringToRGB(base64_string):
    imgdata = base64.b64decode(str(base64_string))
    img = Image.open(io.BytesIO(imgdata))
    opencv_img= cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB)
    opencv_img = cv2.resize(opencv_img, (224,224))
    return opencv_img 

def IMG(data_gen, batch_size):
    feature = np.zeros((batch_size, 224, 224, 3))
    name = np.zeros((batch_size,1))
    while data_gen:
        for i in range(batch_size):
            m = next(b)
            name[i] = m[1][0]
            feature[i] = m[1][1]
        yield (feature, name)

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
spark = SparkSession.builder.getOrCreate()

path_al = '/content/gdrive/MyDrive/out.csv'

In [10]:
df_al = spark.read.csv(path_al, header=False)

In [11]:
df_al.show(200)

+----+--------+---+--------------------+--------------------+--------+--------------------+--------------------+
| _c0|     _c1|_c2|                 _c3|                 _c4|     _c5|                 _c6|                 _c7|
+----+--------+---+--------------------+--------------------+--------+--------------------+--------------------+
|null|m.0107_f|  0|http://getbeatmad...|http://getbeatmad...|FaceId-0|KsQsP3Pumj2B6UE/V...|/9j/4AAQSkZJRgABA...|
|   0|m.0107_f|  1|http://1.bp.blogs...|http://plasticine...|FaceId-0|voT2Po1YFz5oL0E/q...|/9j/4AAQSkZJRgABA...|
|   1|m.0107_f|  2|http://www.magica...|http://www.magica...|FaceId-0|SLTCPgAAqD51oxg/m...|/9j/4AAQSkZJRgABA...|
|   2|m.0107_f|  3|http://2.bp.blogs...|http://lifeoftheb...|FaceId-0|Cu2lPs3MjD57CS0/e...|/9j/4AAQSkZJRgABA...|
|   3|m.0107_f|  4|http://www.magica...|http://www.magica...|FaceId-0|HAqVPjlJfD7x6/8+7...|/9j/4AAQSkZJRgABA...|
|   4|m.0107_f|  5|http://1.bp.blogs...|http://magicalexd...|FaceId-0|TMHaPkrRwz4zyw8/P...|/9j/4

In [12]:
#reading dataframe
df_first = dd.read_csv(path_al)

df = df_first.iloc[:,[1, -1]]
col = list(df.columns)
df = df.rename(columns={col[0]:'label' , col[1]:"encoded pic"})
df.head(600)

,label,encoded pic
0,m.0107_f,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...
1,m.0107_f,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...
2,m.0107_f,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...
3,m.0107_f,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...
4,m.0107_f,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...
...,...,...
595,m.010lz5,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...
596,m.010lz5,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...
597,m.010lz5,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...
598,m.010lz5,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...


In [13]:
df = df.reset_index().compute()

In [14]:
label = df["label"]
type(label)

pandas.core.series.Series

In [16]:
label__ = label.astype('category').cat.codes
# Get one hot encoding of columns B
# one_hot = df.
# Drop column B as it is now encoded
# df = df.drop('B',axis = 1)
# # Join the encoded df
# df = df.join(one_hot)
type(label__)
label__

0          0
1          0
2          0
3          0
4          0
        ... 
4872    1201
4873    1201
4874    1201
4875    1201
4876    1201
Length: 100000, dtype: int16

In [17]:
df__ = df.copy()
df__ = df__.assign(output_label = label__)
df__.head(150)

,index,label,encoded pic,output_label
0,0,m.0107_f,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...,0
1,1,m.0107_f,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...,0
2,2,m.0107_f,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...,0
3,3,m.0107_f,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...,0
4,4,m.0107_f,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...,0
...,...,...,...,...
145,145,m.0109kg,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...,2
146,146,m.0109kg,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...,2
147,147,m.0109kg,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...,2
148,148,m.0109kg,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...,2


In [18]:
_df = df__.iloc[:,[-2,-1]]
_df

,encoded pic,output_label
0,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...,0
1,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...,0
2,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...,0
3,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...,0
4,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...,0
...,...,...
4872,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...,1201
4873,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...,1201
4874,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...,1201
4875,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAQCAwMDAgQDAw...,1201


In [19]:
x_encode = _df["encoded pic"]
label = _df["output_label"]
x_encode
label

0          0
1          0
2          0
3          0
4          0
        ... 
4872    1201
4873    1201
4874    1201
4875    1201
4876    1201
Name: output_label, Length: 100000, dtype: int16

In [20]:
df_len = len(_df)
BATCH_SIZE = df_len // 1201

## Creating Checkpoint

In [21]:
checkpoint_filepath = "MSCELEB_1M.h5"

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

# This callback will stop the training when there is no improvement in the validation loss for three consecutive epochs. 
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [22]:
pretrained_model = tf.keras.applications.MobileNetV3Small(
            include_top=False ,
            weights='imagenet', #tuning10 weights='noisy-student' instead of 'imagenet'
                                #Self-training with Noisy Student improves ImageNet classification https://arxiv.org/abs/1911.04252) 
            #pooling='avg', #tuning1
            input_shape=[224,224, 3]
        )

pretrained_model.trainable = True #tuning8 pretrained_model.trainable = True

model = tf.keras.Sequential([
            pretrained_model, #Base pretrained on ImageNet to extract features from images

            tf.keras.layers.GlobalAveragePooling2D(), ##Attach a new head to act as a classifier
            tf.keras.layers.Dropout(0.3), #tuning3
            tf.keras.layers.Dense(1201, activation='softmax')
        ])

4334752/4334752 [==============================] - 0s 0us/step


In [23]:
model.compile(optimizer='adam'
,loss='sparse_categorical_crossentropy'
,metrics=['accuracy', 'sparse_categorical_accuracy'],
)

## Custom Generator

In [24]:
def IMG(data_gen, batch_size):
    
    img = np.zeros((batch_size, 224, 224, 3))
    name = np.zeros((batch_size,1))
    while data_gen:
      # try:

        for i in range(batch_size):
            m = next(data_gen)
            name[i] = m[1][1]
            img[i] = stringToRGB(m[1][0])
        yield (img, name)

      # except StopIteration:
      #   yield (img, name)

In [32]:
data_gen = _df.iterrows()

In [33]:
a = IMG(data_gen,83)

In [34]:
next(a)[1]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],

In [35]:
BATCH_SIZE = df_len // 1201           # Number of all data // number of classes
STEPS_PER_EPOCH = df_len // BATCH_SIZE
print(STEPS_PER_EPOCH)
print(BATCH_SIZE)
BATCH_SIZE = 32 
STEPS_PER_EPOCH = 100
          # Number of all data // number of classes

1204
83


In [36]:
history = model.fit(IMG(data_gen,BATCH_SIZE) ,steps_per_epoch = STEPS_PER_EPOCH, epochs=2,callbacks=checkpoint)

Epoch 1/2
100/100 [==============================] - ETA: 0s - loss: 4.2839 - accuracy: 0.0100 - sparse_categorical_accuracy: 0.0037

100/100 [==============================] - 7s 70ms/step - loss: 4.2839 - accuracy: 0.0100 - sparse_categorical_accuracy: 0.0037
Epoch 2/2
100/100 [==============================] - ETA: 0s - loss: 4.4503 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 0.0028

100/100 [==============================] - 7s 70ms/step - loss: 4.4503 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 0.0028


In [ ]:
epoch = 2
for m in range(epoch):
  k = 0
  for i in range(df_len // BATCH_SIZE):
      _input = _df.iloc[k:k+BATCH_SIZE,0]
      _label = _df.iloc[k:k+BATCH_SIZE,1]
      lst = []
      k=k+BATCH_SIZE
      for j in _input:
          lst.append(stringToRGB(j))
      print(len(lst))
      _input = np.asarray(lst)

      model.fit(_input, _label)
      
      del lst
      del _input
      del _label

32
1/1 [==============================] - 0s 87ms/step - loss: nan - accuracy: 1.0000 - sparse_categorical_accuracy: 1.0000
32
1/1 [==============================] - 0s 89ms/step - loss: nan - accuracy: 1.0000 - sparse_categorical_accuracy: 1.0000
32
1/1 [==============================] - 0s 76ms/step - loss: nan - accuracy: 1.0000 - sparse_categorical_accuracy: 1.0000
32
1/1 [==============================] - 0s 75ms/step - loss: nan - accuracy: 1.0000 - sparse_categorical_accuracy: 0.3438
32
1/1 [==============================] - 0s 77ms/step - loss: nan - accuracy: 1.0000 - sparse_categorical_accuracy: 0.0000e+00
32
1/1 [==============================] - 0s 79ms/step - loss: nan - accuracy: 1.0000 - sparse_categorical_accuracy: 0.0000e+00
32
1/1 [==============================] - 0s 76ms/step - loss: nan - accuracy: 1.0000 - sparse_categorical_accuracy: 0.0000e+00
32
1/1 [==============================] - 0s 79ms/step - loss: nan - accuracy: 1.0000 - sparse_categorical_accuracy: 0.0

KeyboardInterrupt: ignored